#### Import Python packages 

In [22]:
# Import Python packages 
import cassandra
import re


## Processing of a CSV file 

After processing a single CSV file titled <font color=red>event_datafile_new.csv</font>, is generated within the Workspace directory.
The event_datafile_new.csv contains the following columns: 

- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Create Apache Cassandra infrastructure

#### Creating a Cluster

In [23]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [24]:
# Create a Keyspace 
try :
    session.execute(""" CREATE KEYSPACE IF NOT EXISTS udacity
                    WITH REPLICATION = {
                    'class': 'SimpleStrategy', 'replication_factor': 1 } """)
except Exception as e:
    print("Error: Issue in creating KeySapce ")
    print(e)

#### Set Keyspace

In [25]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print("Error: Issue in setting Keyspace")
    print(e)


### REMEMBER: With Apache Cassandra you model the database tables on the queries you want to run.

Each time you run SELECT queries, remember to execute scripts in the sequence.  `create_tables.py` and `etl.py`files so that event log data is extracted, processed and loaded into the database tables.


### Create queries to ask the following three questions of the processed data 

1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'



### Run SELECT to verify that the data have been inserted into each table

For table music_library, the session_id was used as a partition key because the queries will be filter by this column. 
item_in_session was used as clustering column to help make up a unique key.
That is the reason session_id and item_in_session were selected as a primary key
i.e PRIMARY KEY(session_id, item_in_session)

In [26]:
## Query 1: Give me the artist, song title and song's length in the music app history that was heard 
## during sessionId = 338, and itemInSession = 4¶
## Select statement to validate the data was entered into table correctly

query = "SELECT artist_name, song_title, duration FROM music_library \
         WHERE session_id = 338 AND item_in_session = 4"

try :
    rows = session.execute(query)
    
except Exception as e :
    print(e)
    
for row in rows :
    print (row.artist_name, row.song_title, row.duration)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


For table user_music_library, the user_id and session_id were used as a partition key because the queries will be 
filter by these columns. 
item_in_session was used as clustering column to help make up a unique key.
That is the reason user_id, session_id and item_in_session were selected as a primary key
i.e PRIMARY KEY((user_id, session_id), item_in_session)

Since query also needed song to be sorted descending by item_in_session, **WITH CLUSTERING ORDER BY** was added.
i.e.WITH CLUSTERING ORDER BY (item_in_session DESC)

In [27]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
## Select statement to validate the data was entered into table correctly

select_query = "SELECT session_id, item_in_session, artist_name, song_title,  first_name, last_name  \
                FROM user_music_library \
                WHERE user_id = 10 AND session_id = 182 \
                ORDER BY item_in_session DESC"

try :
    rows = session.execute(select_query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.session_id, row.item_in_session, row.artist_name, row.song_title, row.first_name, row.last_name)


182 3 Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz
182 2 Sebastien Tellier Kilometer Sylvie Cruz
182 1 Three Drives Greece 2000 Sylvie Cruz
182 0 Down To The Bone Keep On Keepin' On Sylvie Cruz


For table user_music_history, the user_id was used as a partition key because the queries will be filter by this column. 
song_title was used as clustering column to help make up a unique key.
That is the reason user_id and song_title were selected as a primary key
i.e PRIMARY KEY (user_id, song_title)

In [28]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
## Select statement to validate the data was entered into table correctly

select_query = "SELECT user_id, first_name, last_name, artist_name  \
                FROM user_music_history \
                WHERE song_title = 'All Hands Against His Own'"

try :
    rows = session.execute(select_query)
except Exception as e:
    print(e)
    

for row in rows:
    print (row.user_id, row.first_name, row.last_name, row.artist_name )
    

29 Jacqueline Lynch The Black Keys
80 Tegan Levine The Black Keys
95 Sara Johnson The Black Keys


### Drop the tables before closing out the sessions

In [29]:
## Drop the tables before closing out the sessions

drop_query1 = "DROP TABLE IF EXISTS udacity.music_library"
try :
    session.execute(drop_query1)
except Exception as e:
    print(e)
    
drop_query2 = "DROP TABLE IF EXISTS udacity.user_music_library"
try :
    session.execute(drop_query2)
except Exception as e:
    print(e)
    
drop_query3 = "DROP TABLE IF EXISTS udacity.user_music_history"
try :
    session.execute(drop_query3)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [30]:
session.shutdown()
cluster.shutdown()